# RECORD LINKAGE: INTRODUCTION AND EXCERCISES

----

This notebook will provide an instruction into Record Linkage using Python. Upon completion of this notebook you will able to apply deterministic and probabilistic linkage techniques to combine data from different sources using commom or non-common identifiers. In addition, this notebook will provide you with a brief overview on data preprocessing needed to find better matches when linking records.

The notebook is structured as follows: 
- [The Principles of Record Linkage](#pricpl)
- [The Importance of Pre-Processing](#prepro)
- [Deterministic Record Linkage](#detlink)
    - [Example Code and Exercises](#code1)
- [Pre-processing of Identifiers](#preproid)
    - [Example Code and Exercises](#code2)   
- [Probabilistic Record Linkage](#problink)
    - [Example Code and Exercises](#code3)
- [References](#ref)

In [ ]:
# Packages we need for all the excercises in this notebook
%pylab inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy
import sklearn
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy
import jellyfish
import re
import recordlinkage as rl

## The Principles of Record Linkage
The goal of record linkage is to determine if pairs of records describe the same identity. This is important for removing duplicates from a data source or joining two separate data sources together. Record linkage also goes by the terms data matching, merge/purge, duplication detection, de-duping, reference matching, entity resolution, disambiguation, co-reference/anaphora in various fields.

There are several approaches to record linkage that include exact matching, rule-based linking and probabilistic linking. An example of exact matching is joining records based on social security number. Rule-based matching involves applying a cascading set of rules that relect the domain knowledge of the records being linked. In probabilistic record linkage, linkage weights are calculated based on records and a threshold is applied to make a decision of whether to link records or not. 

Basic record linkage follows a simple structure which is displayed in the following graph:  

![easylink](./detlink.jpg) 


However, oftentimes no unique entity identifiers are available. In addition, real world data are dirty. For example, we find typographical errors and variations, missing and out-of-date values, different coding schemes. Thus, depending on the data quality and content record linkage often requires pre-processing information before assessing the linkage and thus follows a more complicated structure as outlined in the following:

![complink](./problink.jpg)


In practical applications you will need record linkage techiques to combine information addressing the same entity that is stored in different data sources. Record linkage will also help you to address the quality of different data sources. For example, if one of your databases has missing values you might be able to fill these by liking data from a different source. Overall, the main applications of record linkage are
- Merging two or more data files 
- Identifying the intersection of the two data sets 
- Updating data files (with the data row of the other data files) and imputing missing data

## The Importance of Pre-Processing
Linking records from different data sources comes with different challenges that need to be addressed by the analyst. The analyst must determine whether or not two entities (individuals, businesses, geographical units) on two different files are the same. This determination is not always easy. In most of the cases there is no common uniquely identifing characteristic for a entity. For example, is Bob Miller from New Yor the same person as Bob Miller from Chicago in a given dataset? This detemination has to be executed carefully because consequences of wrong linkages may be substantial (is person X the same person as the person X on the list of identified terrorists). In case of the wage data we will be using later on wrong matches can lead to either under- or oversetimating the productivity of workers. Thus, in case there is no unique identfier available the analyst needs to standardize the records prior to linkage, which is also called pre-processing. 

Pre-processing can be difficult because there are a lot of things to keep in mind. For example, data input errors, such as typos, misspellings, truncation, abbreviations, and missing values need to be corrected.
![complink](./names.jpg)
Literature shows that preprocessing can improve matches. In some situations, 90% of the improvement in matching efficiency may be due to preprocessing. The most common reason why matching projects fail is lack of time and resources for data cleaning.  

### General Workflow
The workflow of data pre-processing is displayed below. After loading the raw data the analyst first needs to define the concept according to which the standardization will be done. This is especially important if you are working with different data that contain common identifiers, however the underling concept is different. In a second step you can apply different methods of parsing through files to identify communalities. Data cleaning also involves making sure all the formats are correct, missing values are handled properly, or that units and names are standardized. we will explore this later in one of the exercises in more detail.

![workflow](./workflow.jpg)

## Deterministic Record Linkage
Deterministic record linkage is the most simple version of record linkage and is also calles rule-based record linkage. It matches data based on a certain number of individual identifiers. Deterministic record linkage is the best way of linking data when the entities in the data to be matched are identified by a common identifier (Social Security Number), combination of common identifiers (Social Security Number and Year), or several representative identifiers with high data quality (name in combination with date of birth and sex and spatial information). In this case we don't need special techniques to link data. In Python we can use the Pandas build in merge function and merge the data on the identifiers, or use SQL to join the databases on these identifiers. 

### Example Code and Exercises
The first question we want to address is **how many of the prisoners who have been released will end up in prison again at a later point in time**. The information we need can be found in two different database tables. The IDOC Admission table and the IDOC Exit table. Luckily these tables can be combined by using an unique idenfier IDOC assigns to the inmates, namely the DOCNBR. For this exercise wedon't ataate need to load the entire database. The relevant information for us to answer our question is the identifier, the date of admission and the date of exit. 
The subsample contain only a reduced amount of information which is important for this class. 

In [ ]:
# Read in data needed from database
# db_name = "appliedda"
# db_host = "10.10.2.10"
# pgsql_connection = psycopg2.connect( host = db_host, database = db_name )
# cur = pgsql_connection.cursor()
pgsql_engine = sqlalchemy.create_engine( "postgresql://10.10.2.10/appliedda" )

In [ ]:
# Select variables needed from the database tables
query = 'SELECT docnbr, curadmdt FROM {table} where curadmyr > 2010;'.format(table="ildoc_admit")
query2 = 'SELECT docnbr, exitda, exitmo, exityr FROM {table} where curadmyr > 2010;'.format(table="ildoc_exit")

In [ ]:
# And load into Pandas data frame
df_adm = pd.read_sql( query, con = pgsql_engine )
df_ext = pd.read_sql( query2, con = pgsql_engine )

In [ ]:
# Let's look at the data frames: admission
print(df_adm.describe())
print(df_adm.dtypes)
df_adm.head()

In [ ]:
# Let's look at the data frames: exit
print(df_ext.describe())
print(df_ext.dtypes)
df_ext.head()

In [ ]:
df_ext.info()

We can see that by only using the docbnr we cannot identify unique matches because one inmate can have multiple admissions and exits. However, we need to make sure that we link the correct admissions and exits. We need to create an identifer for this. For example we can sort the dataframes by admission/exit date and create a marker that contains the counts of admissions/exits. This way the fist record is always marked as 1, the second as 2, ..., the nth as N.

In [ ]:
# Harmonize admission date variables first
df_adm['date_adm'] = pd.to_datetime(df_adm['curadmdt'].astype(str), format='%m%d%Y')
print(df_adm.head())

In [ ]:
# Harmonize exit date variables first
df_ext['exitdt'] = df_ext['exitmo'].astype(int).astype(str).str.rjust(2,'0') + df_ext['exitda'].astype(int).astype(str).str.rjust(2,'0') + df_ext['exityr'].astype(int).astype(str)
print(df_ext.head())

In [ ]:
# We have missings in the exit data (-1), set them to NaN
df_ext = df_ext.replace('-1-1-1', np.NaN)

# Generate date
df_ext['date_ext'] = pd.to_datetime(df_ext['exitdt'].astype(str), format='%m%d%Y') 
print(df_ext.head())

In [ ]:
# Create marker for our rule based matching: 
# Sort by date 
df_adm.sort(['docnbr','date_adm'], ascending=True, inplace=True)
df_ext.sort(['docnbr','date_ext'], ascending=True, inplace=True)

In [ ]:
# And generating counter
df_adm['counter'] = df_adm.groupby('docnbr').cumcount()
print(df_adm.head(10))

df_ext['counter'] = df_ext.groupby('docnbr').cumcount()
print(df_ext.head(10))

Now that we have the selected the information we need and created two dataframes we can merge the dataframes using Pandas (note that this exercise can also be done by joining databases, as outlined in the notebook on databases, session 4). We are using Pandas in this notebook to show you another way to link two different tables. Before lionking the data you should always explore the individual data sets to see if the fields you want to match on are consistent, and to think about the way you wnt to combine the data. Data merges can be performed in different ways (1:1, m:1, 1:m, m:m). 

In [ ]:
# Only keep variables needed for linkage
df_adm = df_adm[['docnbr','counter','date_adm']]
df_ext = df_ext[['docnbr','counter','date_ext']]

**Pandas Syntax used to merge dataframes:**

`DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)[source]`

In [ ]:
# Merge the two datasets on the common identifer
df_adm_ext = pd.merge(left=df_adm,right=df_ext, how='left', left_on=['docnbr','counter'], right_on=['docnbr', 'counter'])
print(df_adm_ext.head(10))

Now that we have the combined data and made sure tht our linkage worked we can turn to answer our question. How many prisoners will end up in prison again after being relased. These are the inmates for which we found 2 or more matches. To get the percentage we can calculate a ratio of these inmates over the number of all inmates in our combined data. Note that the dates do not make sense for some of the cases. Most likely this is due to errors in data collection. This needs to be investigated further. However, for now we just focus on the number of admnissions and exits. 

In [ ]:
# Answer to research question
df_agg = df_adm_ext.groupby('docnbr').agg({'counter':'last'})
df_agg2 = df_agg.groupby('counter').agg({'counter':'count'})
df_agg2['percent'] = df_agg2['counter']/len(df_agg)*100
print(df_agg2.head(10))
df_agg2['percent'].plot(kind='bar')

## Pre-Processing of Identifiers
We saw in the example abouve that even in a simple setting we needed to do some pre=processingbefore linking data. Pre-processing may be more complicated when the data we want to link are more messy. The following section briefly introduces two common ways to clean data.  

### Parsing String Variables

By default, the split method returns a list of strings obtained by splitting the original string on spaces or commas, etc. The split method treats whitespace as delimiting characters, and multiple consecutive whitespace characters are treated like a single delimiter. We can also extract information from strings for example by using regex search commands.

### Regular Expressions - regex
When defining a regular expression search pattern, it is a good idea to start out by writing down, explicitly, in plain English, what you are trying to search for and exactly how you identify when you've found a match.
For example, if we look at an author field formatted as "<last_name> , <first_name> <middle_name>", in plain English, this is how I would explain where to find the last name: "starting from the beginning of the line, take all the characters until you see a comma."

We can build a regular expression that captures this idea from the following components:
- ^ Matches beginning of the line
- . Matches any character
- .+ A modifier that means "match one or more of the preceding expression"

In a regular expression, there are special reserved characters and character classes like those in the list above. Anything that is not a special character or class is just looked for explicitly (for example, a comma is not a special character in regular expressions, so if it is in a regular expression pattern, the regular expression processor will just be looking for a comma in the string, at that point in the pattern).

Note: if you want to actually look for one of these reserved characters, it must be escaped, so that, for example, the expression looks for a literal period, rather than the special regular expression meaning of a period. To escape a reserved character in a regular expression, precede it with a back slash ( "." ).
This results in the regular expression: ^.+,

We start at the beginning of the line ( "^" ), matching any characters ( ".+" ) until we come to the literal character of a comma ( "," ).

In python, to use a regular expression like this to search for matches in a given string, we use the built-in "re" package ( https://docs.python.org/2/library/re.html ), specifically the "re.search()" method. To use "re.search()", pass it first the regular expression you want to use to search, enclosed in quotation marks, and then the string you want to search within:

### Example Code and Excercises
Preprocessing usually involves standardizing names. To protect data privacy of our inmates in the sample we cannot directly use the names of the inmates for this excercise. All identifying information has been hashed. However, before applying the hash we performed name standardization. To still be able to practice pre-processing we will use the firm data instead. We have the name of the firm in the database and all the techniques you would be using to standardize names apply to firm names too. 

For instance sometimes the same company will be listed with different ways for denoting the firm name. Sometimes it might be Name, sometimes Name and legal form etc. To make sure that a linkage on firm names is correct we need to clean the firm names before. In general, this is done be breaking down the name field into various fields which then contain a sinlge part of the name of the firms. Thus, we need to split or parse the information in the fields we have. 

In [ ]:
# Load IDES firm data
query3 = 'SELECT name_legal, name_trade, quarter, year, total_wages FROM {table} LIMIT 1000;'.format(table="il_qcew_employers")
# Save in dataframe
df_firms = pd.read_sql( query3, con = pgsql_engine )

In [ ]:
df_firms.head(10)

In [ ]:
# Split strings
df_firms['fname'] = df_firms.name_trade.str.split(',').str.get(0)    
df_firms['legalf'] = df_firms.name_trade.str.split(' ').str.get(-1)

In [ ]:
# Extract content from strings
df_firms['first'] = df_firms.name_trade.str.extract('^(\w+)\s')

## Probabilistic Record Linkage
Probabilistic record linkage is somewhat different to deterministic record linkage. It takes into account a wieder range of potential identifiers. Identifiers are not unique anymore, which is why this method is also known as fuzzy matching/merging.It is a method that uses properties of variables commom to different datasets to determine the probability that two records refer to the same entity. Examples of the types of data items that might be compared in this method include gender, date of birth, age, and parts of a name.

It computes weights for each idenfier used in the linkage based on the estimated ability to correctly identify a match, or a non-match. Then, by using the estimated weights a probability is calculated that two given records are the same entity. The analyst sets the threshold for this probability to determine when a pairing is defined a match.  

### Fellegi-Sunter Approach
This is a popular method used in probabilisitc record linkage. Let's walk through an example how it works
- Let's assume each person's wage record matches to one person record in the inmate data and we have 100,000 inmates in our inmate data. Then the odds for a match at random are 1:99,999
- M, the reliability is the probability that a commom variable agrees on a matched pair. Approx. 1-error rate
- U, the discriminating power is the probability that a commom variable agrees on a unmatched pair. Approx. the probability of aggreeing by chance
- If first name is the same: m=0.9, u=0.01, ratio: 90:1, this means that the odds for a matchare now: 1:99,999x90:1=1:1,111
- If last name is the same: m=0.9, u=0.04, ratio: 22:1, this means that the odds for a matchare now: 1:1,111x22:1=1:51
- And you can add as many variables as possible, such as sex, age, date of birth, etc as long as they are in both datasets.

### Example Code and Exercises